In [40]:
# !pip install feather-format
import pyarrow.feather as feather
titanic_train = feather.read_feather('titanic_train.feather')
titanic_train.head()
titanic_test = feather.read_feather('titanic_test.feather')
titanic_test.head()

,grupo,id,HomePlanet,CryoSleep,deck,room_num,side,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,fname,lname,gastos,grupo_imp
0,13.0,1.0,Earth,True,G,3.0,S,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,Nelly,Carsoning,0.0,False
1,18.0,1.0,Earth,False,F,4.0,S,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,Lerome,Peckers,2832.0,False
2,19.0,1.0,Europa,True,C,0.0,S,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,Sabih,Unhearfus,0.0,False
3,21.0,1.0,Europa,False,C,1.0,S,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,Meratz,Caltilter,7418.0,False
4,23.0,1.0,Earth,False,F,5.0,S,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,Brence,Harperez,645.0,False


In [41]:
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline

from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.svm import LinearSVC

aplicaremos los siguientes modelos
* TSNE
* K-means
* Linear Regression
* Logistic Regression 
* Random Forest
* Boosted Forest
* PCA
* KNN classifier
* support vector (not in sklearn)

Manual de supervivencia
1. Escoje un modelo
1. Escoje los hiper-parámetros del modelo
1. Prepara el dataset en features matrix y el vector target
1. Divide el dataset en train y test
1. Aplica el algoritmo a tus datos de entrenamiento
1. Aplica el algoritmo a tus datos de prueba
1. Evalúa el modelo
1. Aplica el modelo en nuevos datos

In [44]:
# preparamos nuestro dataset en featura matrix y vector target
Y_train = titanic_train["Transported"]
X_train = titanic_train.drop("Transported", axis=1)
Y_test = titanic_test["Transported"]
X_test = titanic_test.drop("Transported", axis=1)

KeyError: 'Transported'

In [38]:
modelo = LinearRegression(fit_intercept = True)
X = X_train.drop(["lname", "fname", "Destination", "CryoSleep", "HomePlanet", "deck", "side"],axis=1)
modelo.fit(X, Y_train) 

LinearRegression()

In [37]:
print("Coeficientes: {}".format(modelo.coef_))

print("Interceptor: {}".format(modelo.intercept_))

Coeficientes: [ 4.15054910e-06  3.22595149e-02 -4.69275161e-02 -6.22826650e-05
  2.25448704e-02 -1.30119090e-04  9.11147431e-05  5.76985806e-05
 -4.82675242e-05 -4.61801685e-05 -4.95187348e-05]
Interceptor: 0.7178057338335508


In [50]:
## 6. Aplica el algoritmo a tus datos de prueba
y_from_model = modelo.predict(X_test)

## 7. Evalúa el modelo
modelo.score(X_test, y_test) ## Devuelve el R^2x

NameError: name 'X_test' is not defined

In [51]:
## 8. Aplica el modelo en nuevos datos
x_new = np.linspace(-30, 30)

X_new = x_new[:, np.newaxis]
y_new = modelo.predict(X_new)
## Graficamos el modelo y los datos
plt.scatter(x, y)
plt.plot(X_new, y_new, c='red')


NotFittedError: This LinearRegression instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [1]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=1000)
model.fit(X_train, y_train)
y_model = model.predict(X_test)
## Accuracy
from sklearn.metrics import accuracy_score
print("Accuracy: {}".format(accuracy_score(y_test, y_model)))
## Métricas clasicas de Clasificadores
from sklearn import metrics
print(metrics.classification_report(y_model, y_test))

NameError: name 'X_train' is not defined

In [2]:
classifier_names = ['LR',
                    'kNN',
                    'RBF SVM',
                    'Random Forest',
                    'Boosted Trees']
classifiers = [LogisticRegression(random_state=seed,solver='lbfgs'),
               KNeighborsClassifier(5),
               SVC(gamma=2, C=1, random_state=seed),
               RandomForestClassifier(max_depth=5, n_estimators=10, 
                                      max_features=1, random_state=seed),
               GradientBoostingClassifier(n_estimators=10, learning_rate=1.0,
                                          max_depth=5, random_state=seed)]
for model in classifiers:
    model.fit(training_data, training_labels)

NameError: name 'LogisticRegression' is not defined

In [3]:
def test_roc(model, data, labels):
    if hasattr(model, "decision_function"):
        predictions = model.decision_function(data)
    else:
        predictions = model.predict_proba(data)[:,1]
    fpr, tpr, thr = sklearn.metrics.roc_curve(labels, predictions)
    return fpr, tpr, thr

def test_conf(model, data, labels, thr):
    if hasattr(model, "decision_function"):
        predictions = model.decision_function(data)
    else:
        predictions = model.predict_proba(data)[:,1]
    mat = confusion_matrix(labels, predictions>=thr)
    return mat

In [5]:
for name, model in zip(classifier_names, classifiers):
    fpr, tpr, thr = test_roc(model, test_data, test_labels)
    plt.plot(fpr, tpr, label=name)
    print(name+' AUC:{}'.format(sklearn.metrics.auc(fpr, tpr)))
    J = tpr - fpr
    ix = argmax(J)
    best_thresh = thr[ix]
    print(name+' Best Threshold=%f' % (best_thresh))
    mat = test_conf(model, test_data, test_labels, best_thresh)
    print(mat)
    print()

NameError: name 'classifiers' is not defined